In [1]:
#切片，单高斯
import ROOT
import sys

INPUT_FILE = "/eos/user/r/reyu/full_chain_odd/fixparticleHypothesis/selection100Mev/n1500_pion/tracksummary_ckf.root"
TREE_NAME = "tracksummary"

PT_MIN = 0.8
PT_MAX = 12

#CUT = "nTracksTruthVtx > 0 && nTracksRecoVtx >= 2"
#CUT = "vertex_primary == 1 &&  nTracksRecoVtx >= 2 && nTracksTruthVtx > 0"

HIST_X_MIN, HIST_X_MAX = -0.1, 0.1
HIST_BINS = 100

FIT_RANGE_MIN, FIT_RANGE_MAX = -0.05,0.05

f = ROOT.TFile.Open(INPUT_FILE)
t = f.Get(TREE_NAME)

c_slice = ROOT.TCanvas("c_slice", "Slice Inspection", 800, 600)
c_slice.SetGrid() 

ROOT.gStyle.SetOptFit(1111)
ROOT.gStyle.SetOptStat(10) 

hist_name = "h_slice_resT"
hist_title = f"res_ePHI_fit Slice: {PT_MIN:.1f} < t_pT < {PT_MAX:.1f} GeV;mm;Entries / bin"
h_slice = ROOT.TH1D(hist_name, hist_title, HIST_BINS, HIST_X_MIN, HIST_X_MAX)

full_cut = f"t_pT > {PT_MIN} && t_pT < {PT_MAX} "
print("-" * 60)
print(f"{full_cut}")
print("-" * 60)
t.Draw(f"res_ePHI_fit >> {hist_name}", full_cut)

entries = h_slice.GetEntries()
print(f" {int(entries)} vertex")

fit_func = ROOT.TF1("gaus", "gaus", FIT_RANGE_MIN, FIT_RANGE_MAX)
fit_func.SetLineColor(ROOT.kRed) 
fit_func.SetLineWidth(3)


print(f"fit in  [{FIT_RANGE_MIN}, {FIT_RANGE_MAX}] ")

fit_status = h_slice.Fit(fit_func, "R") 
#fit_status = h_slice.Fit("gaus", "", "", FIT_RANGE_MIN, FIT_RANGE_MAX)
h_slice.SetMarkerStyle(20) 
h_slice.SetMarkerSize(0.8)
h_slice.SetLineColor(ROOT.kBlack)

h_slice.Draw("E")
c_slice.Update()

st = h_slice.FindObject("stats")
if st:
    st.SetX1NDC(0.65) 
    st.SetX2NDC(0.90) 
    st.SetY1NDC(0.60) 
    st.SetY2NDC(0.90) 
#output_filename = f"SliceFit_{PT_MIN}to{PT_MAX}GeV.png"
#c_slice.SaveAs(output_filename)

c_slice.Draw()


------------------------------------------------------------
t_pT > 0.8 && t_pT < 12 
------------------------------------------------------------
 1704001 vertex
fit in  [-0.05, 0.05] 
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =       110921
NDf                       =           47
Edm                       =  1.04728e-07
NCalls                    =           79
Constant                  =       729103   +/-   815.078     
Mean                      = -4.17307e-06   +/-   1.38132e-06 
Sigma                     =   0.00174331   +/-   1.37591e-06  	 (limited)


In [21]:
c_slice.Draw()

In [6]:
#切片双高斯
import ROOT
import sys

INPUT_FILE = "/eos/user/r/reyu/full_chain_odd/fixparticleHypothesis/selection100Mev/n1500_pion/tracksummary_ckf.root"
TREE_NAME = "tracksummary"

PT_MIN = 0.8
PT_MAX = 12

HIST_X_MIN, HIST_X_MAX = -0.01, 0.01
HIST_BINS = 100

FIT_RANGE_MIN, FIT_RANGE_MAX = -0.005,0.005

f = ROOT.TFile.Open(INPUT_FILE)
t = f.Get(TREE_NAME)

c_slice = ROOT.TCanvas("c_slice", "Slice Inspection", 800, 600)
c_slice.SetGrid() 

ROOT.gStyle.SetOptFit(1111)
ROOT.gStyle.SetOptStat(10) 

hist_name = "h_slice_resT"
hist_title = f"res_eTHETA_fit Slice: {PT_MIN:.1f} < t_pT < {PT_MAX:.1f} GeV;mm;Entries / bin"
h_slice = ROOT.TH1D(hist_name, hist_title, HIST_BINS, HIST_X_MIN, HIST_X_MAX)

full_cut = f"t_pT > {PT_MIN} && t_pT < {PT_MAX} "
print("-" * 60)
print(f"{full_cut}")
print("-" * 60)
t.Draw(f"res_eTHETA_fit >> {hist_name}", full_cut)

entries = h_slice.GetEntries()

#fit_func = ROOT.TF1("gaus", "gaus", FIT_RANGE_MIN, FIT_RANGE_MAX)

func_str = "[0]*exp(-0.5*((x-[1])/[2])^2) + [3]*exp(-0.5*((x-[1])/[4])^2)"
fit_func = ROOT.TF1("double_gaus", func_str, FIT_RANGE_MIN, FIT_RANGE_MAX)
total_amp = h_slice.GetMaximum()
mean_est  = h_slice.GetMean()
rms_est   = h_slice.GetRMS()

# [0] Core 高度：假设占总高度的 80%
fit_func.SetParameter(0, total_amp * 0.8)

# [1] Mean：设为直方图的均值
fit_func.SetParameter(1, mean_est)

# [2] Core Sigma：设得比 RMS 小一点 (比如 1/3 RMS)
fit_func.SetParameter(2, rms_est * 0.3)
fit_func.SetParLimits(2, 0.001, rms_est) # 限制 Core Sigma 不能太大

# [3] Tail 高度：假设占总高度的 20%
fit_func.SetParameter(3, total_amp * 0.2)

# [4] Tail Sigma：设得比 RMS 大一点
fit_func.SetParameter(4, rms_est * 1.5)
fit_func.SetLineColor(ROOT.kRed) 
fit_func.SetLineWidth(3)


print(f"fit in  [{FIT_RANGE_MIN}, {FIT_RANGE_MAX}] ")

fit_status = h_slice.Fit(fit_func, "R") 
#fit_status = h_slice.Fit("gaus", "", "", FIT_RANGE_MIN, FIT_RANGE_MAX)
h_slice.SetMarkerStyle(20) 
h_slice.SetMarkerSize(0.8)
h_slice.SetLineColor(ROOT.kBlack)

h_slice.Draw("E")
c_slice.Update()

st = h_slice.FindObject("stats")
if st:
    st.SetX1NDC(0.65) 
    st.SetX2NDC(0.90) 
    st.SetY1NDC(0.60) 
    st.SetY2NDC(0.90) 
#output_filename = f"SliceFit_{PT_MIN}to{PT_MAX}GeV.png"
#c_slice.SaveAs(output_filename)

c_slice.Draw()


------------------------------------------------------------
t_pT > 0.8 && t_pT < 12 
------------------------------------------------------------
fit in  [-0.005, 0.005] 
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      35991.1
NDf                       =           46
Edm                       =  8.29469e-06
NCalls                    =          119
p0                        =       209356   +/-   504.498     
p1                        =  1.95883e-07   +/-   4.09342e-07 
p2                        =  0.000235704                      	 (fixed)
p3                        =       100062   +/-   249.952     
p4                        =  0.000835695   +/-   1.06688e-06 


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_slice


In [7]:
c_slice.Draw()

In [5]:
#时间分bin选择，单高斯两次拟合
import ROOT
import array
import sys

# ================= 配置区域 =================
FILE_pion = "/eos/user/r/reyu/full_chain_odd/fixparticleHypothesis/selection100Mev/n1500_pion/tracksummary_ckf.root"  
FILE_none    = "/eos/user/r/reyu/full_chain_odd/fixparticleHypothesis/selection100Mev/n1500_none/tracksummary_ckf.root"     
FILE_proton    = "/eos/user/r/reyu/full_chain_odd/fixparticleHypothesis/selection100Mev/n1500_proton/tracksummary_ckf.root"         

TREE_NAME = "tracksummary"
#var = "res_eLOC0_fit"
#CUT = "nTracksRecoVtx >= 2 && nTracksTruthVtx > 0"

# 拟合范围设置
FIT_MIN, FIT_MAX = -35, 35
NSIGMA = 3  #二次拟合的核心区间宽度 (Mean +/- NSIGMA * Sigma)


# bin_edges = [
#      0.8,1.2,1.6 ,2.0, 3.0, 4.0, 5.0, 
#     12
# ]

bin_edges = [
     0.8,1.5,2.2 ,3.0, 3.7, 4.4, 5.1, 
    12
]
pt_bins = array.array('d', bin_edges)
n_bins = len(pt_bins) - 1

# ===========================================

def get_sigma_hist(filename, label, color, marker):
    f = ROOT.TFile.Open(filename)
    if not f or f.IsZombie():
        print(f"Error opening file: {filename}")
        return None
        
    t = f.Get(TREE_NAME)
    
    # 1. 创建 2D 直方图并填充
    h2_name = f"h2_{label}"
    h2 = ROOT.TH2D(h2_name, "", n_bins, pt_bins, 100, FIT_MIN, FIT_MAX)
   # print(f"[{label}] Filling histogram... Cut: {CUT}")
    #t.Draw(f"resT:sqrt(sumPt2) >> {h2_name}", CUT, "goff")
    t.Draw(f"res_eT_fit:t_pT >> {h2_name}", "", "goff")
    # 2. 创建用于存放 Sigma 结果的 1D 直方图
    h_sigma = ROOT.TH1D(f"sigma_{label}", "", n_bins, pt_bins)
    
    # 3. 手动循环每一个 Bin 进行二次拟合
    print(f"[{label}] Performing iterative Gaussian fitting...")
  
    for i in range(1, h2.GetNbinsX() + 1):
        # 取出当前 Bin 的 Y 轴投影 (resT 分布)
        h_slice = h2.ProjectionY(f"slice_{label}_{i}", i, i)
        
        # 如果统计量太少，跳过
        if h_slice.GetEntries() < 5:
            continue
       
        # --- 第一次拟合 (全范围) ---
        # "Q"=Quiet, "N"=NoDraw, "0"=Do not store graphics
        fit_res1 = h_slice.Fit("gaus", "QNS", "", FIT_MIN, FIT_MAX)
         
        # 检查拟合是否成功
        if int(fit_res1) != 0: continue 

        #func1 = h_slice.GetFunction("gaus")
        #if not func1: continue
        
        mu_rough =fit_res1.Parameter(1)
        sigma_rough = fit_res1.Parameter(2)
        #mu_rough = func1.GetParameter(1)
        #sigma_rough = func1.GetParameter(2)
        #print(sigma_rough)
        # --- 定义核心范围 (Core Range) ---
        fit_range_min = mu_rough - NSIGMA * sigma_rough
        fit_range_max = mu_rough + NSIGMA * sigma_rough
       
        
        
        # --- 第二次拟合 (核心范围) ---
        # "R" = Use Range specified in function arguments
        fit_res2 = h_slice.Fit("gaus", "QNSR", "", fit_range_min, fit_range_max)
        
        # 如果第二次拟合失败，回退使用第一次的结果，或者跳过
#         if int(fit_res2) != 0:
#             final_sigma = sigma_rough
#             final_error = func1.GetParError(2)
#         else:
#             func2 = h_slice.GetFunction("gaus")
#             final_sigma = func2.GetParameter(2)
#             final_error = func2.GetParError(2)
            
        # 4. 将结果填入 h_sigma
        final_sigma = fit_res2.Parameter(2)
        final_error = fit_res2.ParError(2)
        h_sigma.SetBinContent(i, final_sigma)
        h_sigma.SetBinError(i, final_error)
        
        # 清理内存
        del h_slice

    # 设置样式
    h_sigma.SetDirectory(0) # 让直方图脱离文件管理，防止文件关闭后消失
    f.Close()
    
    h_sigma.SetTitle("")
    h_sigma.SetLineColor(color)
    h_sigma.SetMarkerColor(color)
    h_sigma.SetMarkerStyle(marker)
    h_sigma.SetMarkerSize(1.2)
    h_sigma.SetLineWidth(2)
    
    return h_sigma

# ================= 主程序 =================


h_default = get_sigma_hist(FILE_pion, "Pion", ROOT.kBlack, 21)
h_mass    = get_sigma_hist(FILE_none, "MassHypo", ROOT.kRed, 20)
h_proton = get_sigma_hist(FILE_proton, "proton", ROOT.kBlue, 20) # 如果需要，取消注释

c = ROOT.TCanvas("c", "Comparison", 800, 800)

# --- 上方 Pad (Sigma 对比) ---
pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 1.0)
pad1.SetBottomMargin(0.02) 
pad1.SetLogx()              
pad1.Draw()
pad1.cd()

h_default.GetYaxis().SetTitle(" mean_eT/ns")
h_default.GetYaxis().SetTitleSize(0.05)
h_default.GetYaxis().SetTitleOffset(1.0)
h_default.GetXaxis().SetLabelSize(0)

# 自动调整 Y 轴范围
max_val = h_default.GetMaximum()
if h_mass: max_val = max(max_val, h_mass.GetMaximum())
h_default.GetYaxis().SetRangeUser(0, 10) # 稍微留点空间

h_default.Draw("E1")      
if h_mass: h_mass.Draw("E1 SAME")
if h_proton: h_proton.Draw("E1 SAME")

leg = ROOT.TLegend(0.35, 0.55, 0.7, 0.85)
leg.AddEntry(h_default, "Pion Hypothesis", "lp")
leg.AddEntry(h_mass,    "Truth Hypothesis",    "lp")
leg.AddEntry(h_proton, "Proton Hypothesis", "lp")
leg.SetBorderSize(0)
leg.Draw()

lat = ROOT.TLatex()
lat.SetNDC()
lat.SetTextSize(0.04)
lat.DrawLatex(0.15, 0.75, "ttbar-pu200")
lat.DrawLatex(0.15, 0.80, "events=1500")
#lat.DrawLatex(0.15, 0.85, CUT)

# --- 下方 Pad (Ratio) ---
c.cd()
pad2 = ROOT.TPad("pad2", "pad2", 0, 0.0, 1, 0.3)
pad2.SetTopMargin(0.02)    
pad2.SetBottomMargin(0.3)  
pad2.SetLogx()             
pad2.SetGridy()            
pad2.Draw()
pad2.cd()

if h_mass:
    h_ratio = h_mass.Clone("h_ratio")
    h_ratio.Divide(h_default)

    h_ratio.SetTitle("")
    h_ratio.GetYaxis().SetTitle("Ratio (Truth / Pion)")
    h_ratio.GetXaxis().SetTitle("t_pT [GeV]")

    h_ratio.GetXaxis().SetTitleSize(0.12)
    h_ratio.GetXaxis().SetLabelSize(0.10)
    h_ratio.GetXaxis().SetTitleOffset(1.1)
    
    h_ratio.GetYaxis().SetTitleSize(0.10)
    h_ratio.GetYaxis().SetLabelSize(0.10)
    h_ratio.GetYaxis().SetTitleOffset(0.5)
    h_ratio.GetYaxis().SetNdivisions(505) 

    h_ratio.GetYaxis().SetRangeUser(0, 2) # Ratio 范围
    h_ratio.Draw("E1")

    line = ROOT.TLine(bin_edges[0], 1, bin_edges[-1], 1)
    line.SetLineStyle(2)
    line.SetLineColor(ROOT.kGray+2)
    line.Draw()

#c.SaveAs("Comparison_Sigma_IterativeFit.png")
c.Draw()
print("Saved as Comparison_Sigma_IterativeFit.png")


[Pion] Performing iterative Gaussian fitting...
[MassHypo] Performing iterative Gaussian fitting...
[proton] Performing iterative Gaussian fitting...
Saved as Comparison_Sigma_IterativeFit.png


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


In [64]:
print(pt_bins)

array('d', [0.4, 0.8, 1.2, 1.6, 2.0, 3.0, 4.0, 5.0, 7.0, 10.0, 15.0, 20.0, 30.0, 50.0, 100.0, 200.0])


In [4]:
#时间分bin选择，双高斯
import ROOT
import array
import sys

# ================= 配置区域 =================
FILE_pion = "/eos/user/r/reyu/full_chain_odd/fixparticleHypothesis/selection100Mev/n1500_pion/tracksummary_ckf.root"  # 基准文件
FILE_none    = "/eos/user/r/reyu/full_chain_odd/fixparticleHypothesis/selection100Mev/n1500_none/tracksummary_ckf.root"     # 测试文件 (加了质量假设)
FILE_proton    = "/eos/user/r/reyu/full_chain_odd/fixparticleHypothesis/selection100Mev/n1500_proton/tracksummary_ckf.root"     # 测试文件     
   

TREE_NAME = "tracksummary"
#var = "res_eLOC0_fit"
#CUT = "nTracksRecoVtx >= 2 && nTracksTruthVtx > 0"

# 拟合范围设置
FIT_MIN, FIT_MAX = -35, 35


# bin_edges = [
#     0.4, 0.8,1.2,1.6 ,2.0, 3.0, 4.0, 5.0, 
#     7.0, 10.0, 15.0, 20.0, 30.0, 
#     50.0, 100.0, 200.0
# ]
bin_edges = [0.8,1.5,2.2 ,3.0, 3.7, 4.4, 5.1, 12]

pt_bins = array.array('d', bin_edges)
n_bins = len(pt_bins) - 1

# ===========================================

def get_sigma_hist(filename, label, color, marker):
    f = ROOT.TFile.Open(filename)
    if not f or f.IsZombie():
        print(f"Error opening file: {filename}")
        return None
        
    t = f.Get(TREE_NAME)
    
    # 1. 创建 2D 直方图并填充
    h2_name = f"h2_{label}"
    h2 = ROOT.TH2D(h2_name, "", n_bins, pt_bins, 100, FIT_MIN, FIT_MAX)
   # print(f"[{label}] Filling histogram... Cut: {CUT}")
    #t.Draw(f"resT:sqrt(sumPt2) >> {h2_name}", CUT, "goff")
    t.Draw(f"res_eT_fit:t_pT >> {h2_name}", "", "goff")
    # 2. 创建用于存放 Sigma 结果的 1D 直方图
    h_sigma = ROOT.TH1D(f"sigma_{label}", "", n_bins, pt_bins)
    
    
    print(f"[{label}] Performing 2 Gaussian fitting...")
  
    for i in range(1, h2.GetNbinsX() + 1):
        # 取出当前 Bin 的 Y 轴投影 (resT 分布)
        h_slice = h2.ProjectionY(f"slice_{label}_{i}", i, i)
        
        func_str = "[0]*exp(-0.5*((x-[1])/[2])^2) + [3]*exp(-0.5*((x-[1])/[4])^2)"
        fit_func = ROOT.TF1("double_gaus", func_str, FIT_MIN, FIT_MAX)
        total_amp = h_slice.GetMaximum()
        mean_est  = h_slice.GetMean()
        rms_est   = h_slice.GetRMS()

        # [0] Core 高度：假设占总高度的 80%
        fit_func.SetParameter(0, total_amp * 0.8)

        # [1] Mean：设为直方图的均值
        fit_func.SetParameter(1, mean_est)

        # [2] Core Sigma：设得比 RMS 小一点 (比如 1/3 RMS)
        fit_func.SetParameter(2, rms_est * 0.3)
        fit_func.SetParLimits(2, 0.001, rms_est) # 限制 Core Sigma 不能太大

        # [3] Tail 高度：假设占总高度的 20%
        fit_func.SetParameter(3, total_amp * 0.2)

        # [4] Tail Sigma：设得比 RMS 大一点
        fit_func.SetParameter(4, rms_est * 1.5)
        fit_func.SetLineColor(ROOT.kRed) 
        fit_func.SetLineWidth(3)


       # print(f"fit in  [{FIT_RANGE_MIN}, {FIT_RANGE_MAX}] ")

        fit_res1 = h_slice.Fit(fit_func, "RS") 
        # 检查拟合是否成功
        if int(fit_res1) != 0: continue 

        #func1 = h_slice.GetFunction("gaus")
        #if not func1: continue
        
        sigma1 =fit_res1.Parameter(2)
        #print(sigma1)
        sigma2 = fit_res1.Parameter(4)
        sigma1error = fit_res1.ParError(2)
        #print(sigma1error)
        sigma2error = fit_res1.ParError(4)
        h_sigma.SetBinContent(i, sigma1)
        h_sigma.SetBinError(i, sigma1error)
        
        
        
        
        
        # 清理内存
        del h_slice

    # 设置样式
    h_sigma.SetDirectory(0) # 让直方图脱离文件管理，防止文件关闭后消失
    f.Close()
    
    h_sigma.SetTitle("")
    h_sigma.SetLineColor(color)
    h_sigma.SetMarkerColor(color)
    h_sigma.SetMarkerStyle(marker)
    h_sigma.SetMarkerSize(1.2)
    h_sigma.SetLineWidth(2)
    
    return h_sigma

# ================= 主程序 =================


h_default = get_sigma_hist(FILE_pion, "Pion", ROOT.kBlack, 21)
h_mass    = get_sigma_hist(FILE_none, "MassHypo", ROOT.kRed, 20)
h_proton = get_sigma_hist(FILE_proton, "proton", ROOT.kBlue, 20) # 如果需要，取消注释

c = ROOT.TCanvas("c", "Comparison", 800, 800)

# --- 上方 Pad (Sigma 对比) ---
pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 1.0)
pad1.SetBottomMargin(0.02) 
pad1.SetLogx()              
pad1.Draw()
pad1.cd()

h_default.GetYaxis().SetTitle(" res_eLOC1_fit/mm")
h_default.GetYaxis().SetTitleSize(0.05)
h_default.GetYaxis().SetTitleOffset(1.0)
h_default.GetXaxis().SetLabelSize(0)

# 自动调整 Y 轴范围
max_val = h_default.GetMaximum()
if h_mass: max_val = max(max_val, h_mass.GetMaximum())
h_default.GetYaxis().SetRangeUser(0, 30) # 稍微留点空间

h_default.Draw("E1")      
if h_mass: h_mass.Draw("E1 SAME")
if h_proton: h_proton.Draw("E1 SAME")

leg = ROOT.TLegend(0.35, 0.55, 0.7, 0.85)
leg.AddEntry(h_default, "Pion Hypothesis", "lp")
leg.AddEntry(h_mass,    "Truth Hypothesis",    "lp")
leg.AddEntry(h_proton, "Proton Hypothesis", "lp")
# leg.SetBorderSize(0)
leg.Draw()

lat = ROOT.TLatex()
lat.SetNDC()
lat.SetTextSize(0.04)
lat.DrawLatex(0.15, 0.75, "ttbar-pu200")
lat.DrawLatex(0.15, 0.80, "events=1500")
#lat.DrawLatex(0.15, 0.85, CUT)

#--- 下方 Pad (Ratio) ---
c.cd()
pad2 = ROOT.TPad("pad2", "pad2", 0, 0.0, 1, 0.3)
pad2.SetTopMargin(0.02)    
pad2.SetBottomMargin(0.3)  
pad2.SetLogx()             
pad2.SetGridy()            
pad2.Draw()
pad2.cd()

if h_mass:
    h_ratio = h_mass.Clone("h_ratio")
    h_ratio.Divide(h_default)

    h_ratio.SetTitle("")
    h_ratio.GetYaxis().SetTitle("Ratio (Truth / Pion)")
    h_ratio.GetXaxis().SetTitle("t_pT [GeV]")

    h_ratio.GetXaxis().SetTitleSize(0.12)
    h_ratio.GetXaxis().SetLabelSize(0.10)
    h_ratio.GetXaxis().SetTitleOffset(1.1)
    
    h_ratio.GetYaxis().SetTitleSize(0.10)
    h_ratio.GetYaxis().SetLabelSize(0.10)
    h_ratio.GetYaxis().SetTitleOffset(0.5)
    h_ratio.GetYaxis().SetNdivisions(505) 

    h_ratio.GetYaxis().SetRangeUser(0.9, 1.1) # Ratio 范围
    h_ratio.Draw("E1")

    line = ROOT.TLine(bin_edges[0], 1, bin_edges[-1], 1)
    line.SetLineStyle(2)
    line.SetLineColor(ROOT.kGray+2)
    line.Draw()

#c.SaveAs("Comparison_Sigma_IterativeFit.png")
c.Draw()
print("Saved as Comparison_Sigma_IterativeFit.png")


[Pion] Performing 2 Gaussian fitting...
[MassHypo] Performing 2 Gaussian fitting...
[proton] Performing 2 Gaussian fitting...
Saved as Comparison_Sigma_IterativeFit.png
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      1199.43
NDf                       =           95
Edm                       =   1.1105e-06
NCalls                    =          315
p0                        =        14076   +/-   847.511     
p1                        =      1.30147   +/-   0.0121225   
p2                        =       9.5711   +/-   0.139768     	 (limited)
p3                        =      10449.7   +/-   861.627     
p4                        =      13.7344   +/-   0.178198    
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      153.942
NDf                       =           95
Edm                       =  7.07624e-06
NCalls                    =          326
p0                        =      7595.4

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


In [22]:
c.Draw()

In [24]:
#其余径迹参数研究，d0，z0，双gauss
import ROOT
import array
import sys

# ================= 配置区域 =================
FILE_pion = "/eos/user/r/reyu/full_chain_odd/fixparticleHypothesis/selection100Mev/n1500_pion/tracksummary_ckf.root"  # 基准文件
FILE_none    = "/eos/user/r/reyu/full_chain_odd/fixparticleHypothesis/selection100Mev/n1500_none/tracksummary_ckf.root"     # 测试文件 (加了质量假设)
FILE_proton    = "/eos/user/r/reyu/full_chain_odd/fixparticleHypothesis/selection100Mev/n1500_proton/tracksummary_ckf.root"     # 测试文件     

TREE_NAME = "tracksummary"
#var = "res_eLOC0_fit"
#CUT = "nTracksRecoVtx >= 2 && nTracksTruthVtx > 0"

# 拟合范围设置
FIT_MIN, FIT_MAX = -0.03, 0.03



# bin_edges = [
#      0.8,1.2,1.6 ,2.0, 3.0, 4.0, 5.0, 12]
bin_edges = [0.8,1.5,2.2 ,3.0, 3.7, 4.4, 5.1, 12]

pt_bins = array.array('d', bin_edges)
n_bins = len(pt_bins) - 1

# ===========================================

def get_sigma_hist(filename, label, color, marker):
    f = ROOT.TFile.Open(filename)
    if not f or f.IsZombie():
        print(f"Error opening file: {filename}")
        return None
        
    t = f.Get(TREE_NAME)
    
    # 1. 创建 2D 直方图并填充
    h2_name = f"h2_{label}"
    h2 = ROOT.TH2D(h2_name, "", n_bins, pt_bins, 100, FIT_MIN, FIT_MAX)
   # print(f"[{label}] Filling histogram... Cut: {CUT}")
    #t.Draw(f"resT:sqrt(sumPt2) >> {h2_name}", CUT, "goff")
    t.Draw(f"res_ePHI_fit:t_pT >> {h2_name}", "", "goff")
    # 2. 创建用于存放 Sigma 结果的 1D 直方图
    h_sigma = ROOT.TH1D(f"sigma_{label}", "", n_bins, pt_bins)
    
    
    print(f"[{label}] Performing 2 Gaussian fitting...")
  
    for i in range(1, h2.GetNbinsX() + 1):
        # 取出当前 Bin 的 Y 轴投影 (resT 分布)
        h_slice = h2.ProjectionY(f"slice_{label}_{i}", i, i)
        
        func_str = "[0]*exp(-0.5*((x-[1])/[2])^2) + [3]*exp(-0.5*((x-[1])/[4])^2)"
        fit_func = ROOT.TF1("double_gaus", func_str, FIT_MIN, FIT_MAX)
        total_amp = h_slice.GetMaximum()
        mean_est  = h_slice.GetMean()
        rms_est   = h_slice.GetRMS()

        # [0] Core 高度：假设占总高度的 80%
        fit_func.SetParameter(0, total_amp * 0.8)

        # [1] Mean：设为直方图的均值
        fit_func.SetParameter(1, mean_est)

        # [2] Core Sigma：设得比 RMS 小一点 (比如 1/3 RMS)
        fit_func.SetParameter(2, rms_est * 0.3)
        fit_func.SetParLimits(2, 0.001, rms_est) # 限制 Core Sigma 不能太大

        # [3] Tail 高度：假设占总高度的 20%
        fit_func.SetParameter(3, total_amp * 0.2)

        # [4] Tail Sigma：设得比 RMS 大一点
        fit_func.SetParameter(4, rms_est * 1.5)
        fit_func.SetLineColor(ROOT.kRed) 
        fit_func.SetLineWidth(3)


       # print(f"fit in  [{FIT_RANGE_MIN}, {FIT_RANGE_MAX}] ")

        fit_res1 = h_slice.Fit(fit_func, "RS") 
        # 检查拟合是否成功
        if int(fit_res1) != 0: continue 

        #func1 = h_slice.GetFunction("gaus")
        #if not func1: continue
        
        sigma1 =fit_res1.Parameter(1)
        #print(sigma1)
        sigma2 = fit_res1.Parameter(4)
        sigma1error = fit_res1.ParError(1)
        #print(sigma1error)
        sigma2error = fit_res1.ParError(4)
        h_sigma.SetBinContent(i, sigma1)
        h_sigma.SetBinError(i, sigma1error)
        
        
        
        
        
        # 清理内存
        del h_slice

    # 设置样式
    h_sigma.SetDirectory(0) # 让直方图脱离文件管理，防止文件关闭后消失
    f.Close()
    
    h_sigma.SetTitle("")
    h_sigma.SetLineColor(color)
    h_sigma.SetMarkerColor(color)
    h_sigma.SetMarkerStyle(marker)
    h_sigma.SetMarkerSize(1.2)
    h_sigma.SetLineWidth(2)
    
    return h_sigma

# ================= 主程序 =================


h_default = get_sigma_hist(FILE_pion, "Pion", ROOT.kBlack, 21)
h_mass    = get_sigma_hist(FILE_none, "MassHypo", ROOT.kRed, 20)
h_proton = get_sigma_hist(FILE_proton, "proton", ROOT.kBlue, 20) # 如果需要，取消注释

c = ROOT.TCanvas("c", "Comparison", 800, 800)

# --- 上方 Pad (Sigma 对比) ---
pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 1.0)
pad1.SetBottomMargin(0.02) 
pad1.SetLogx()              
pad1.Draw()
pad1.cd()

h_default.GetYaxis().SetTitle(" res_ePHi_fit/mm")
h_default.GetYaxis().SetTitleSize(0.05)
h_default.GetYaxis().SetTitleOffset(1.0)
h_default.GetXaxis().SetLabelSize(0)

# 自动调整 Y 轴范围
max_val = h_default.GetMaximum()
if h_mass: max_val = max(max_val, h_mass.GetMaximum())
h_default.GetYaxis().SetRangeUser(-0.01, 0.01) # 稍微留点空间

h_default.Draw("E1")      
if h_mass: h_mass.Draw("E1 SAME")
if h_proton: h_proton.Draw("E1 SAME")

leg = ROOT.TLegend(0.35, 0.55, 0.7, 0.85)
leg.AddEntry(h_default, "Pion Hypothesis", "lp")
leg.AddEntry(h_mass,    "Truth Hypothesis",    "lp")
leg.AddEntry(h_proton, "Proton Hypothesis", "lp")
# leg.SetBorderSize(0)
leg.Draw()

lat = ROOT.TLatex()
lat.SetNDC()
lat.SetTextSize(0.04)
lat.DrawLatex(0.15, 0.75, "ttbar-pu200")
lat.DrawLatex(0.15, 0.80, "events=1500")
#lat.DrawLatex(0.15, 0.85, CUT)

#--- 下方 Pad (Ratio) ---
c.cd()
pad2 = ROOT.TPad("pad2", "pad2", 0, 0.0, 1, 0.3)
pad2.SetTopMargin(0.02)    
pad2.SetBottomMargin(0.3)  
pad2.SetLogx()             
pad2.SetGridy()            
pad2.Draw()
pad2.cd()

if h_mass:
    h_ratio = h_mass.Clone("h_ratio")
    h_ratio.Divide(h_default)

    h_ratio.SetTitle("")
    h_ratio.GetYaxis().SetTitle("Ratio (Truth / Pion)")
    h_ratio.GetXaxis().SetTitle("t_pT [GeV]")

    h_ratio.GetXaxis().SetTitleSize(0.12)
    h_ratio.GetXaxis().SetLabelSize(0.10)
    h_ratio.GetXaxis().SetTitleOffset(1.1)
    
    h_ratio.GetYaxis().SetTitleSize(0.10)
    h_ratio.GetYaxis().SetLabelSize(0.10)
    h_ratio.GetYaxis().SetTitleOffset(0.5)
    h_ratio.GetYaxis().SetNdivisions(505) 

    h_ratio.GetYaxis().SetRangeUser(-0.1, 2) # Ratio 范围
    h_ratio.Draw("E1")

    line = ROOT.TLine(bin_edges[0], 1, bin_edges[-1], 1)
    line.SetLineStyle(2)
    line.SetLineColor(ROOT.kGray+2)
    line.Draw()

#c.SaveAs("Comparison_Sigma_IterativeFit.png")
c.Draw()
print("Saved as Comparison_Sigma_IterativeFit.png")


[Pion] Performing 2 Gaussian fitting...
[MassHypo] Performing 2 Gaussian fitting...
[proton] Performing 2 Gaussian fitting...
Saved as Comparison_Sigma_IterativeFit.png
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      14565.2
NDf                       =           95
Edm                       =  7.32475e-06
NCalls                    =          182
p0                        =       113985   +/-   215.822     
p1                        = -6.48516e-06   +/-   1.99484e-06 
p2                        =   0.00148257   +/-   3.81958e-06  	 (limited)
p3                        =      15925.6   +/-   179.75      
p4                        =   0.00432344   +/-   1.78617e-05 
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      6307.48
NDf                       =           95
Edm                       =  1.14344e-05
NCalls                    =          182
p0                        =      72030.

Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Warning in <Fit>: Abnormal termination of minimization.
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Warning in <Fit>: Abnormal termination of minimization.
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will 

In [14]:
c.Draw()

In [55]:
#顶点分辨率和均值，t,x,y,z
import ROOT
import array
import sys

# ================= 配置区域 =================
FILE_pion   = "/eos/user/r/reyu/full_chain_odd/fixparticleHypothesis/selection100Mev/n1500_pion/vertex_amvf_truth_time/performance_vertexing.root" 
FILE_none   = "/eos/user/r/reyu/full_chain_odd/fixparticleHypothesis/selection100Mev/n1500_none/vertex_amvf_truth_time/performance_vertexing.root"      
FILE_proton = "/eos/user/r/reyu/full_chain_odd/fixparticleHypothesis/selection100Mev/n1500_proton/vertex_amvf_truth_time/performance_vertexing.root"      

TREE_NAME = "vertexing"
CUT = "nTracksRecoVtx >= 2 && nTracksTruthVtx > 0 && sqrt(sumPt2)< 100"
#CUT = " nTracksTruthVtx > 0"
# 拟合范围设置
FIT_MIN, FIT_MAX = -10.0, 10
NSIGMA = 3.0  #二次拟合的核心区间宽度 (Mean +/- NSIGMA * Sigma)

# bin_edges = [
#      0.4, 0.8,1.2,1.6 ,2.0, 3.0, 4.0, 5.0, 
#     7.0, 10.0, 15.0, 20.0, 30.0, 
#     50.0, 100.0, 200.0
# ]
bin_edges = [1.5,2.2 ,3.0, 3.7, 4.4, 5.1, 12,20,50,100]
pt_bins = array.array('d', bin_edges)
n_bins = len(pt_bins) - 1

# ===========================================

def get_sigma_hist(filename, label, color, marker):
    f = ROOT.TFile.Open(filename)
    if not f or f.IsZombie():
        print(f"Error opening file: {filename}")
        return None
        
    t = f.Get(TREE_NAME)
    
    # 1. 创建 2D 直方图并填充
    h2_name = f"h2_{label}"
    h2 = ROOT.TH2D(h2_name, "", n_bins, pt_bins, 100, FIT_MIN, FIT_MAX)
    print(f"[{label}] Filling histogram... Cut: {CUT}")
    t.Draw(f"resT:sqrt(sumPt2) >> {h2_name}", CUT, "goff")
    
    # 2. 创建用于存放 Sigma 结果的 1D 直方图
    h_sigma = ROOT.TH1D(f"sigma_{label}", "", n_bins, pt_bins)
    
    # 3. 手动循环每一个 Bin 进行二次拟合
    print(f"[{label}] Performing iterative Gaussian fitting...")
  
    for i in range(1, h2.GetNbinsX() + 1):
        # 取出当前 Bin 的 Y 轴投影 (resT 分布)
        h_slice = h2.ProjectionY(f"slice_{label}_{i}", i, i)
        
        # 如果统计量太少，跳过
        if h_slice.GetEntries() < 20:
            continue
       
        # --- 第一次拟合 (全范围) ---
        # "Q"=Quiet, "N"=NoDraw, "0"=Do not store graphics
        fit_res1 = h_slice.Fit("gaus", "QNS", "", FIT_MIN, FIT_MAX)
         
        # 检查拟合是否成功
        if int(fit_res1) != 0: continue 

        #func1 = h_slice.GetFunction("gaus")
        #if not func1: continue
        
        mu_rough =fit_res1.Parameter(1)
        sigma_rough = fit_res1.Parameter(2)
        #mu_rough = func1.GetParameter(1)
        #sigma_rough = func1.GetParameter(2)
       
        # --- 定义核心范围 (Core Range) ---
        fit_range_min = mu_rough - NSIGMA * sigma_rough
        fit_range_max = mu_rough + NSIGMA * sigma_rough
       
        
        
        # --- 第二次拟合 (核心范围) ---
        # "R" = Use Range specified in function arguments
        fit_res2 = h_slice.Fit("gaus", "QNSR", "", fit_range_min, fit_range_max)
        
        # 如果第二次拟合失败，回退使用第一次的结果，或者跳过
#         if int(fit_res2) != 0:
#             final_sigma = sigma_rough
#             final_error = func1.GetParError(2)
#         else:
#             func2 = h_slice.GetFunction("gaus")
#             final_sigma = func2.GetParameter(2)
#             final_error = func2.GetParError(2)
            
        # 4. 将结果填入 h_sigma
        final_mu = fit_res2.Parameter(1)
        final_error = fit_res2.ParError(1)
        h_sigma.SetBinContent(i, final_mu)
        h_sigma.SetBinError(i, final_error)
        
        # 清理内存
        del h_slice

    # 设置样式
    h_sigma.SetDirectory(0) # 让直方图脱离文件管理，防止文件关闭后消失
    f.Close()
    
    h_sigma.SetTitle("")
    h_sigma.SetLineColor(color)
    h_sigma.SetMarkerColor(color)
    h_sigma.SetMarkerStyle(marker)
    h_sigma.SetMarkerSize(1.2)
    h_sigma.SetLineWidth(2)
    
    return h_sigma

# ================= 主程序 =================


h_default = get_sigma_hist(FILE_pion, "Pion", ROOT.kBlack, 21)
h_mass    = get_sigma_hist(FILE_none, "MassHypo", ROOT.kRed, 20)
h_proton = get_sigma_hist(FILE_proton, "proton", ROOT.kBlue, 20) # 如果需要，取消注释

c = ROOT.TCanvas("c", "Comparison", 800, 800)

# --- 上方 Pad (Sigma 对比) ---
pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 1.0)
pad1.SetBottomMargin(0.02) 
pad1.SetLogx()              
pad1.Draw()
pad1.cd()

h_default.GetYaxis().SetTitle(" mean(resT) [ns]")
h_default.GetYaxis().SetTitleSize(0.05)
h_default.GetYaxis().SetTitleOffset(1.0)
h_default.GetXaxis().SetLabelSize(0)

# 自动调整 Y 轴范围
max_val = h_default.GetMaximum()
if h_mass: max_val = max(max_val, h_mass.GetMaximum())
h_default.GetYaxis().SetRangeUser(-10, 10) # 稍微留点空间

h_default.Draw("E1")      
if h_mass: h_mass.Draw("E1 SAME")
if h_proton: h_proton.Draw("E1 SAME")

leg = ROOT.TLegend(0.35, 0.55, 0.7, 0.8)
leg.AddEntry(h_default, "Pion Hypothesis", "lp")
leg.AddEntry(h_mass,    "Truth Hypothesis",    "lp")
leg.AddEntry(h_proton, "Proton Hypothesis", "lp")
leg.SetBorderSize(0)
leg.Draw()

lat = ROOT.TLatex()
lat.SetNDC()
lat.SetTextSize(0.04)
lat.DrawLatex(0.15, 0.75, "ttbar-pu200")
lat.DrawLatex(0.15, 0.80, "events=1500")
lat.DrawLatex(0.15, 0.85, CUT)

# --- 下方 Pad (Ratio) ---
c.cd()
pad2 = ROOT.TPad("pad2", "pad2", 0, 0.0, 1, 0.3)
pad2.SetTopMargin(0.02)    
pad2.SetBottomMargin(0.3)  
pad2.SetLogx()             
pad2.SetGridy()            
pad2.Draw()
pad2.cd()

if h_mass:
    h_ratio = h_mass.Clone("h_ratio")
    h_ratio.Divide(h_default)

    h_ratio.SetTitle("")
    h_ratio.GetYaxis().SetTitle("Ratio (Truth / Pion)")
    h_ratio.GetXaxis().SetTitle("Vertex #sum p_{T} [GeV]")

    h_ratio.GetXaxis().SetTitleSize(0.12)
    h_ratio.GetXaxis().SetLabelSize(0.10)
    h_ratio.GetXaxis().SetTitleOffset(1.1)
    
    h_ratio.GetYaxis().SetTitleSize(0.10)
    h_ratio.GetYaxis().SetLabelSize(0.10)
    h_ratio.GetYaxis().SetTitleOffset(0.5)
    h_ratio.GetYaxis().SetNdivisions(505) 

    h_ratio.GetYaxis().SetRangeUser(-1, 2) # Ratio 范围
    h_ratio.Draw("E1")

    line = ROOT.TLine(bin_edges[0], 1, bin_edges[-1], 1)
    line.SetLineStyle(2)
    line.SetLineColor(ROOT.kGray+2)
    line.Draw()

#c.SaveAs("Comparison_Sigma_IterativeFit.png")
c.Draw()
print("Saved as Comparison_Sigma_IterativeFit.png")


[Pion] Filling histogram... Cut: nTracksRecoVtx >= 2 && nTracksTruthVtx > 0 && sqrt(sumPt2)< 100
[Pion] Performing iterative Gaussian fitting...
[MassHypo] Filling histogram... Cut: nTracksRecoVtx >= 2 && nTracksTruthVtx > 0 && sqrt(sumPt2)< 100
[MassHypo] Performing iterative Gaussian fitting...
[proton] Filling histogram... Cut: nTracksRecoVtx >= 2 && nTracksTruthVtx > 0 && sqrt(sumPt2)< 100
[proton] Performing iterative Gaussian fitting...
Saved as Comparison_Sigma_IterativeFit.png


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
